# Introduction

This notebook compares the excitation energies from fully QM calculations 
(done at the CIS/cc-pVDZ) level of theory to those from four 
different QM/EFP embedding schemes:
- zero or PE: traditional polarizable embedding, contains electrostatics and polarization terms
-  zero_screen or PE+S: polarizable embedding with additional charge-penetration screen correction
-  zero_exrep or PE+XR: polarizable embedding with parametrized exchange-repulsion
-  semiz or PE+SXR: polarizable embedding with the screen correction and parametrized exchange-repulsion

# Load modules and define constants

In [1]:
import numpy as np
import pandas as pd
import math

# Bokeh modules
#from bokeh.models import HoverTool
from bokeh.layouts import column
from bokeh.core.properties import value
from bokeh.io import show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.transform import dodge
from bokeh.models import Span
from bokeh.models.markers import Triangle
from bokeh.models import FuncTickFormatter#, LinearAxis
#from bokeh.models.tickers import FixedTicker
#from bokeh.embed import file_html
#from bokeh.resources import CDN
output_notebook()


Loading BokehJS ...

In [2]:
#conversion factors
hartree_to_ev = 27.2114

#colors
lg = '#77945F'
lb = '#5E8E94'
lo = '#FFA567'
lp = '#945977'
dg = '#517F2B'
db = '#2A757F'
do = '#FF771B'
dp = '#7F2653'

# Read, combine and clean data

In [3]:
# read gamess data
gamess = pd.read_csv('gamess_results.csv')
gamess['state_energy_2'] = gamess['state_energy'].round(3)
gamess = gamess.drop('state_energy', axis=1)
gamess = gamess.rename(columns={'state_energy_2' : 'state_energy'})

# read qchem data
chtr = pd.read_csv('NTO_analysis.csv')
chtr = chtr.round(3)

In [4]:
# create master by merging both dataframes
master = gamess.merge(chtr, how='left',
                      left_on = ['state_energy', 'geo', 'state'],
                      right_on = ['state_energy', 'geo', 'state'])

In [5]:
mycols = \
['qm', 'qm_num',  
 'zero', 'zero_num', 'zero_exrep', 'zero_exrep_num', 
 'zero_screen', 'zero_screen_num', 'semiz', 'semiz_num', 
 'transition_type', 'geo', 
 'state','state_energy', 'h+_chtr', 'e-_chtr','multiplicity']

# clean master
master = master[mycols]

In [6]:
# check for missing data from NTO analysis
master[(master['h+_chtr'].isnull())|(master['e-_chtr'].isnull())]

,qm,qm_num,zero,zero_num,zero_exrep,zero_exrep_num,zero_screen,zero_screen_num,semiz,semiz_num,transition_type,geo,state,state_energy,h+_chtr,e-_chtr,multiplicity
86,7.686286,7,7.719074,7.0,7.718485,7.0,7.728986,11.0,7.718859,7.0,pp,1o,7.0,-682.256,NaN,NaN,NaN
134,6.018623,3,6.015644,3.0,6.018737,3.0,6.013847,4.0,6.018192,3.0,np,1u,3.0,-662.807,NaN,NaN,NaN
159,8.710100,9,8.845230,9.0,8.744201,9.0,8.906362,11.0,8.795512,9.0,np,4d,9.0,-616.312,NaN,NaN,NaN
312,5.058692,1,5.052363,1.0,5.046570,1.0,5.056970,3.0,5.046711,1.0,pp,1d,1.0,-702.186,NaN,NaN,NaN


In [7]:
# create function to add missing values. 
# The values were determined from manually checking and comparing 
# the fullqm GAMESS output and the QChem NTO output

def add_missing_chtr(chtr_state, master_state, geo, master):

    chtr_id = chtr.index[(chtr['geo'] == geo) & (chtr['state'] == chtr_state)].tolist()[0]
    chtr_vals = chtr.loc[chtr_id,['h+_chtr','e-_chtr', 'multiplicity' ]].tolist()
    master_id = master.index[(master['geo'] == geo) & (master['state'] == master_state)].tolist()[0]
    master.loc[master_id, ['h+_chtr','e-_chtr','multiplicity']] = chtr_vals
    
    return(master)

In [8]:
master = add_missing_chtr(1, 1, '1d', master)
master = add_missing_chtr(7, 7, '1o', master)
master = add_missing_chtr(3, 3, '1u', master)
master = add_missing_chtr(9, 9, '4d', master)

In [9]:
# make sure missing values from NTO were fixed
master[(master['h+_chtr'].isnull())|(master['e-_chtr'].isnull())]

,qm,qm_num,zero,zero_num,zero_exrep,zero_exrep_num,zero_screen,zero_screen_num,semiz,semiz_num,transition_type,geo,state,state_energy,h+_chtr,e-_chtr,multiplicity


# Calculate Errors and Summarize Data

In [10]:
# Compute the energy as fullqm exc. energy - QM/EFP exc. energy
qmefp_cols = ['zero', 'zero_screen', 'zero_exrep', 'semiz']
for col in qmefp_cols:
    master.loc[:,f'{col}_error'] =  master.loc[:,f'{col}'] - master.loc[:,'qm']
    master.loc[:,f'{col}_abs_error'] = abs(master.loc[:,f'{col}'] - master.loc[:,'qm'])

In [11]:
# summarize data

schemes_names = {'zero' : 'PE', 'zero_screen' : 'PE+S',
                 'zero_exrep' : 'PE+XR', 'semiz' : 'PE+SXR'}

print(f'Total number of excitations considered: {master.shape[0]}')
print(f'Total number of geometries considered: {master["geo"].unique().shape[0]}')
for scheme in schemes_names.keys():
      missing_ee = master[master[f'{scheme}_error'].isnull()].shape[0]
      scheme_name = schemes_names[scheme]
      print(f'Missing excitations for scheme {scheme_name}: {missing_ee}')
      

Total number of excitations considered: 387
Total number of geometries considered: 37
Missing excitations for scheme PE: 12
Missing excitations for scheme PE+S: 89
Missing excitations for scheme PE+XR: 7
Missing excitations for scheme PE+SXR: 17


In [12]:
data_stats = {}
data_stats['scheme'] = ['zero', 'zero_exrep']
data_stats['mean'] = [master['zero_error'].mean(), 
                      master['zero_exrep_error'].mean()]
data_stats['std'] = [np.std(master['zero_error']), 
                     np.std(master['zero_exrep_error'])]
data_stats['median'] = [master['zero_error'].median(), 
                        master['zero_exrep_error'].median()]
data_stats['abs_mean'] = [master['zero_abs_error'].mean(), 
                          master['zero_exrep_abs_error'].mean()]

In [27]:
print('Summary Statistics for PE (zero) scheme and PE+XR (zero_exrep) scheme')
pd.DataFrame(data_stats).round(3)

Summary Statistics for PE (zero) scheme and PE+XR (zero_exrep) scheme


,scheme,mean,std,median,abs_mean
0,zero,0.024,0.052,0.011,0.035
1,zero_exrep,0.004,0.034,0.005,0.023


In [28]:
print('Absolut mean error for PE (zero) and PE+XR (zero_exrep) schemes based on transition type')
round(master[master['transition_type']!='pr'][['transition_type', 'zero_abs_error','zero_exrep_abs_error']]\
      .groupby('transition_type').agg('mean'), 3)

Absolut mean error for PE (zero) and PE+XR (zero_exrep) schemes based on transition type


,zero_abs_error,zero_exrep_abs_error
transition_type,,
np,0.069,0.022
pp,0.018,0.018


# PLOTS

In [15]:
## THIS CODE WAS TAKEN FROM BOKEH DOCUMENTATION

master_copy = master.rename(columns={'zero_error' : 'a', 
                                     'zero_screen_error' : 'b',
                                     'zero_exrep_error' : 'c',
                                     'semiz_error' : 'd'})

master_melt = pd.melt(master_copy[['qm_num', 'geo', 'a', 'b', 'c', 'd']], 
                      id_vars=["qm_num", "geo"], 
                      var_name="scheme", 
                      value_name="error")

colors1 = [lo, lg, lb, lp]
colors2 = [do, dg, db, dp]
label_dict = {'a' : 'PE', 
              'b' : 'PE+S', 
              'c' : 'PE+XR', 
              'd': 'PE+SXR'}

groups = master_melt[['scheme', 'error']].groupby('scheme')
q1 = groups.quantile(q=0.25)
q2 = groups.quantile(q=0.5)
q3 = groups.quantile(q=0.75)
iqr = q3 - q1
upper = q3 + 1.5*iqr
lower = q1 - 1.5*iqr
mean = groups.mean()
cats = list(groups.groups.keys())

# find the outliers for each category
def outliers(group):
    cat = group.name
    return group[(group.error > upper.loc[cat]['error']) |\
                 (group.error < lower.loc[cat]['error'])]\
                ['error']
out = groups.apply(outliers).dropna()

# prepare outlier data for plotting, we need coordinates for every outlier.
if not out.empty:
    outx = []
    outy = []
    for keys in out.index:
        if out.loc[keys[0]].loc[keys[1]] < 0.145:
            outx.append(str(keys[0]))
            outy.append(out.loc[keys[0]].loc[keys[1]])

            p = figure(x_range=cats, plot_height=350, plot_width=500, y_range=(-0.15,0.15))

# if no outliers, shrink lengths of stems to be no longer than
# the minimums or maximums
qmin = groups.quantile(q=0.00)
qmax = groups.quantile(q=1.00)
upper.error = [min([x,y]) for (x,y) in \
                     zip(list(qmax.loc[:,'error']),upper.error)]
lower.error = [max([x,y]) for (x,y) in \
                     zip(list(qmin.loc[:,'error']),lower.error)]
# stems
p.segment(cats, upper.error, cats, q3.error, line_color="black", line_width=1)
p.segment(cats, lower.error, cats, q1.error, line_color="black", line_width=1)

# boxes
p.vbar(cats, 0.7, q2.error, q3.error, fill_color=colors1,
       line_color="black", fill_alpha=1, line_width=1)
p.vbar(cats, 0.7, q1.error, q2.error, fill_color=colors2,
       line_color="black", fill_alpha=1, line_width=1)

# whiskers (almost-0 height rects simpler than segments)
p.rect(cats, lower.error, 0.2, 0.0001, line_color="black", fill_color="black")
p.rect(cats, upper.error, 0.2, 0.0001, line_color="black", fill_color="black")

# outliers
if not out.empty:
    p.circle(outx, outy, size=8, color="gray", fill_alpha=0.6)

# horizontal line at 0
hline = Span(location=0.0, dimension='width', line_color='gray',
             line_width=1, line_dash='dashed')
p.renderers.extend([hline])

# means
p.triangle(cats, mean['error'].values, color="black", size=8)

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.xaxis.axis_label = "scheme"
p.yaxis.axis_label = "error (eV)"
p.outline_line_color = None
p.xaxis.formatter = FuncTickFormatter(code="""
    var labels = %s;
    return labels[tick];
""" % label_dict)

show(p)